In [24]:
import polars as pl
from pathlib import Path

In [25]:
data_dir = Path("../data/")

In [26]:
train = pl.read_csv(data_dir / "01_raw/train.csv")

In [136]:
oil = pl.scan_csv(data_dir / "01_raw/oil.csv").select([
pl.col("date").str.strptime(pl.Date, "%Y-%m-%d"),
    pl.col("dcoilwtico").alias("price"),
]).filter(pl.col("price").is_not_null()).collect()

In [290]:
cols_to_scale = ["price"]
oil_scaler = StandardScaler()


oil_scaled = oil.with_columns(
    pl.DataFrame(oil_scaler.fit_transform(
        oil.select(pl.col(cols_to_scale)).to_numpy()
    ), schema=cols_to_scale).select(pl.all().suffix("_scaled"))
)

In [291]:
oil_arr = oil_scaled.select("price_scaled").to_numpy()

In [310]:
from sklearn.model_selection import train_test_split

In [311]:
y_train, y_val = train_test_split(oil_arr, train_size=0.8)

In [76]:
import torch
from torch import tensor

In [109]:
from torch.utils.data import Dataset, DataLoader

In [292]:
class OilPricesDataset(Dataset):
    def __init__(self, array, batch_length):
        self.array = torch.from_numpy(array).type(torch.float32)
        self.batch_length = batch_length
        
    def __len__(self):
        return self.array.shape[0] - self.batch_length
    
    def __getitem__(self, idx):
        id_arr = idx + self.batch_length
        return self.array[id_arr - self.batch_length: id_arr], self.array[id_arr]

In [313]:
train_size = 5
batch_size = 20

train_loader = DataLoader(OilPricesDataset(y_train, train_size), batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(OilPricesDataset(y_val, train_size), batch_size=batch_size, shuffle=True, drop_last=True)

In [294]:
import torch
import torch.nn as nn
import torch.optim as optim

model = nn.Sequential(
            nn.Linear(train_size * batch_size, batch_size),)
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.MSELoss()
n_epochs = 100

In [322]:
class LSTMForecaster(nn.Module):


    def __init__(self, n_features, n_hidden, n_outputs, sequence_len, n_lstm_layers=1, n_deep_layers=10, use_cuda=False, dropout=0.2):
        '''
        n_features: number of input features (1 for univariate forecasting)
        n_hidden: number of neurons in each hidden layer
        n_outputs: number of outputs to predict for each training example
        n_deep_layers: number of hidden dense layers after the lstm layer
        sequence_len: number of steps to look back at for prediction
        dropout: float (0 < dropout < 1) dropout ratio between dense layers
        '''
        super().__init__()

        self.n_lstm_layers = n_lstm_layers
        self.nhid = n_hidden
        self.use_cuda = use_cuda # set option for device selection

        # LSTM Layer
        self.lstm = nn.LSTM(n_features,
                            n_hidden,
                            num_layers=n_lstm_layers,
                            batch_first=True) # As we have transformed our data in this way

        # first dense after lstm
        self.fc1 = nn.Linear(n_hidden * sequence_len, n_hidden) 
        # Dropout layer 
        self.dropout = nn.Dropout(p=dropout)

        # Create fully connected layers (n_hidden x n_deep_layers)
        dnn_layers = []
        for i in range(n_deep_layers):
          # Last layer (n_hidden x n_outputs)
          if i == n_deep_layers - 1:
            dnn_layers.append(nn.ReLU())
            dnn_layers.append(nn.Linear(nhid, n_outputs))
          # All other layers (n_hidden x n_hidden) with dropout option
          else:
            dnn_layers.append(nn.ReLU())
            dnn_layers.append(nn.Linear(nhid, nhid))
            if dropout:
                dnn_layers.append(nn.Dropout(p=dropout))
        # compile DNN layers
        self.dnn = nn.Sequential(*dnn_layers)

    def forward(self, x):

        # Initialize hidden state
        hidden_state = torch.zeros(self.n_lstm_layers, x.shape[0], self.nhid)
        cell_state = torch.zeros(self.n_lstm_layers, x.shape[0], self.nhid)

        # move hidden state to device
        if self.use_cuda:
            hidden_state = hidden_state.to(device)
            cell_state = cell_state.to(device)

        self.hidden = (hidden_state, cell_state)

        # Forward Pass
        x, h = self.lstm(x, self.hidden) # LSTM
        x = self.dropout(x.contiguous().view(x.shape[0], -1)) # Flatten lstm out 
        x = self.fc1(x) # First Dense
        return self.dnn(x) # Pass forward through fully connected DNN.

In [319]:
nhid = 50 # Number of nodes in the hidden layer
n_dnn_layers = 5 # Number of hidden fully connected layers
nout = 1 # Prediction Window
sequence_len = train_size # Training Window

# Number of features (since this is a univariate timeseries we'll set
# this to 1 -- multivariate analysis is coming in the future)
ninp = 1

# Device selection (CPU | GPU)
USE_CUDA = torch.cuda.is_available()
device = 'cuda' if USE_CUDA else 'cpu'

# Initialize the model
model = LSTMForecaster(ninp, nhid, nout, sequence_len, n_deep_layers=n_dnn_layers, use_cuda=USE_CUDA).to(device)


In [320]:
# Set learning rate and number of epochs to train over
lr = 4e-4
n_epochs = 20

# Initialize the loss function and optimizer
criterion = nn.MSELoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

In [321]:
# Lists to store training and validation losses
t_losses, v_losses = [], []
# Loop over epochs
for epoch in range(n_epochs):
    train_loss, valid_loss = 0.0, 0.0

    # train step
    model.train()
    # Loop over train dataset
    for x, y in train_loader:
        optimizer.zero_grad()
        # move inputs to device
        x = x.to(device)
        y  = y.squeeze().to(device)
        # Forward Pass
        preds = model(x).squeeze()
        loss = criterion(preds, y) # compute batch loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    epoch_loss = train_loss / len(train_loader)
    t_losses.append(epoch_loss)

    # validation step
    model.eval()
    # Loop over validation dataset
    for x, y in val_loader:
        with torch.no_grad():
            x, y = x.to(device), y.squeeze().to(device)
            preds = model(x).squeeze()
            error = criterion(preds, y)
        valid_loss += error.item()
    valid_loss = valid_loss / len(val_loader)
    v_losses.append(valid_loss)

    print(f'{epoch} - train: {epoch_loss}, valid: {valid_loss}')


plot_losses(t_losses, v_losses)

0 - train: 0.9996164298575857, valid: 1.0098348639228127
1 - train: 1.0000891465207804, valid: 0.9994138533418829
2 - train: 0.9988052559935529, valid: 1.0242966196753762
3 - train: 0.9935230081496031, valid: 1.0299051187255166
4 - train: 0.9952625917351764, valid: 1.0124791643836282
5 - train: 0.999161668445753, valid: 1.0083116238767451
6 - train: 0.9938781896363134, valid: 1.0196453983133489
7 - train: 0.997894841691722, valid: 1.0191043940457432
8 - train: 0.9997080098027769, valid: 1.025453887202523
9 - train: 0.999415617922078, valid: 1.0203527754003352
10 - train: 0.9967144131660461, valid: 1.014822244644165
11 - train: 0.9990392508714095, valid: 1.0340031656351956
12 - train: 0.9929921588172084, valid: 1.0236997116695752
13 - train: 0.9922063467295273, valid: 1.0434598976915532
14 - train: 0.986374477977338, valid: 1.0312170819802717
15 - train: 0.9958538127982098, valid: 1.0256436196240513
16 - train: 0.9890517566515051, valid: 1.0354358174584128
17 - train: 0.9917583323043325

NameError: name 'plot_losses' is not defined

In [295]:
for epoch in range(n_epochs):
    for img, label in oil_dl:
        out = model(img.view(-1).squeeze())
        loss = loss_fn(out, label.view(-1))
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.831569
Epoch: 1, Loss: 0.379933
Epoch: 2, Loss: 0.375677
Epoch: 3, Loss: 0.174501
Epoch: 4, Loss: 0.105528
Epoch: 5, Loss: 0.044817
Epoch: 6, Loss: 0.061509
Epoch: 7, Loss: 0.027500
Epoch: 8, Loss: 0.021195
Epoch: 9, Loss: 0.008202
Epoch: 10, Loss: 0.006289
Epoch: 11, Loss: 0.007742
Epoch: 12, Loss: 0.010367
Epoch: 13, Loss: 0.006189
Epoch: 14, Loss: 0.005130
Epoch: 15, Loss: 0.011133
Epoch: 16, Loss: 0.006254
Epoch: 17, Loss: 0.007326
Epoch: 18, Loss: 0.003874
Epoch: 19, Loss: 0.004246
Epoch: 20, Loss: 0.005532
Epoch: 21, Loss: 0.007784
Epoch: 22, Loss: 0.007530
Epoch: 23, Loss: 0.002169
Epoch: 24, Loss: 0.008489
Epoch: 25, Loss: 0.005591
Epoch: 26, Loss: 0.005705
Epoch: 27, Loss: 0.009285
Epoch: 28, Loss: 0.004243
Epoch: 29, Loss: 0.007454
Epoch: 30, Loss: 0.002393
Epoch: 31, Loss: 0.002804
Epoch: 32, Loss: 0.008110
Epoch: 33, Loss: 0.003389
Epoch: 34, Loss: 0.010065
Epoch: 35, Loss: 0.004284
Epoch: 36, Loss: 0.010099
Epoch: 37, Loss: 0.005928
Epoch: 38, Loss: 0.007

In [304]:
oil_scaler.inverse_transform(out.view(1,...).detach().numpy())

TypeError: view(): argument 'size' must be tuple of ints, but found element of type ellipsis at pos 2